### Importing dependencies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import sklearn.datasets
from sklearn.model_selection import train_test_split

### Data Collection and Analysis

In [ ]:
breast_cancer_dataset = sklearn.datasets.load_breast_cancer()

In [ ]:
print(breast_cancer_dataset)
print("keys: " , breast_cancer_dataset.keys())

In [ ]:
data_frame = pd.DataFrame(breast_cancer_dataset['data'], columns=breast_cancer_dataset['feature_names'])

In [ ]:
data_frame.head()

In [ ]:
data_frame['target'] = breast_cancer_dataset['target']

In [ ]:
data_frame.shape
data_frame.info()

In [ ]:
data_frame.describe()

In [ ]:
data_frame['target'].value_counts()

1 -> Benign \
0 -> Malignant 